In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../pyFiles/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

# from configobj import ConfigObj
# from configobj import flatten_errors
# from validate import Validator
    
import logging

# from analysisParameters import loadParameters
# from loadDataE1 import loadSessionDict

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os

init_notebook_mode(connected=True)


In [2]:
# sessionDict = projectTrajectories(sessionDict)
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
import pickle

file = open('../sessionFiles.pickle', 'rb')
allSessions = pickle.load(file)
file.close()

sessionDict = allSessions[0]

In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

In [5]:
mergedRaw = pd.merge( sessionDict['rawExpUnity'], sessionDict['rawExpGaze'].reset_index(), on ='pupilTimestamp',how='outer',sort=True)
interpRaw = mergedRaw.interpolate(method='linear',downcast='infer')
interpRaw

,ballColliderRadius,ballMeshRadius,ballPos_x,ballPos_y,ballPos_z,ballRot_x,ballRot_y,ballRot_z,ballVel_x,ballVel_y,...,gaze_normal0_z,gaze_normal1_x,gaze_normal1_y,gaze_normal1_z,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,norm_pos_x,norm_pos_y,world_index
0,0.5,0.120000,-6.590693,1.833734,18.000000,0.0,0.0,0.0,7.023213,4.293863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,0.120000,-6.572692,1.844460,17.955013,0.0,0.0,0.0,7.023213,4.257530,...,0.991611,-0.135368,0.108406,0.984847,-133.431046,124.446552,1194.402385,0.427440,0.409768,16238.0
2,0.5,0.120000,-6.554692,1.855185,17.910027,0.0,0.0,0.0,7.023213,4.221196,...,0.991333,-0.135368,0.108406,0.984847,-136.053943,127.849936,1213.863320,0.427200,0.408786,16239.0
3,0.5,0.120000,-6.536691,1.865911,17.865040,0.0,0.0,0.0,7.023213,4.184863,...,0.991333,-0.136532,0.109131,0.984605,-133.440682,125.240509,1184.605920,0.426825,0.408432,16239.0
4,0.5,0.120000,-6.517156,1.877341,17.816218,0.0,0.0,0.0,7.023213,4.157613,...,0.991333,-0.137697,0.109856,0.984363,-130.827420,122.631082,1155.348519,0.426451,0.408078,16239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7931,0.5,0.034197,5.401500,2.786755,-11.450830,0.0,0.0,0.0,6.445778,1.078642,...,0.993877,-0.143545,0.088850,0.985646,-88.720143,72.665572,821.588019,0.429854,0.423412,17345.0
7932,0.5,0.034197,5.423625,2.790409,-11.502003,0.0,0.0,0.0,6.445778,1.042309,...,0.993877,-0.144796,0.087705,0.985567,-87.716571,70.992615,807.687722,0.429461,0.423880,17345.0
7933,0.5,0.034197,5.445749,2.794062,-11.553177,0.0,0.0,0.0,6.445778,1.005975,...,0.993614,-0.144796,0.087705,0.985567,-90.312719,73.303645,825.582292,0.428947,0.423105,17345.0
7934,0.5,0.034197,5.467874,2.797716,-11.604350,0.0,0.0,0.0,6.445778,0.969642,...,0.993614,-0.130947,0.161910,0.975105,-97.855293,126.350910,956.759308,0.433011,0.390150,17345.0


In [6]:
def calcAzEl(row,varName):
    
    x=row['{}_x'.format(varName)]
    y=row['{}_y'.format(varName)]
    z=row['{}_z'.format(varName)]
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 

In [7]:
interpRaw['worldVidTimeSec'] = np.array(interpRaw['pupilTimestamp'] - interpRaw['pupilTimestamp'].iloc[0])
interpRaw['worldVidTimeSec'].tail(10)

7926    36.867783
7927    36.871685
7928    36.871706
7929    36.877798
7930    36.881718
7931    36.882204
7932    36.885655
7933    36.889612
7934    36.892885
7935    36.893618
Name: worldVidTimeSec, dtype: float64

In [8]:
# list(df2)

In [9]:
df2 = interpRaw

ballAzEl = df2.apply(lambda arbitraryRowName: calcAzEl(arbitraryRowName,'ballPos'),axis=1)
df2['ball_az'],df2['ball_el'] = zip(*ballAzEl)

azEl = df2.apply(lambda arbitraryRowName: calcAzEl(arbitraryRowName,'gaze_point_3d'),axis=1)
df2['gazeInWorldCam_az'],df2['gazeInWorldCam_el'] = zip(*azEl)

df2['deltaT'] = df2['worldVidTimeSec'].diff()

df2['velInWorldCam_az'] = df2['gazeInWorldCam_az'].diff() / df2['deltaT']
df2['velInWorldCam_el'] = df2['gazeInWorldCam_el'].diff() / df2['deltaT']
df2['velInWorldCam'] = np.abs(np.sqrt(df2['gazeInWorldCam_az'].diff()**2  + df2['gazeInWorldCam_el'].diff()**2) / df2['deltaT'])

df2['ballVelInWorldCam_az'] = df2['ball_az'].diff() / df2['deltaT']
df2['ballVelInWorldCam_el'] = df2['ball_el'].diff() / df2['deltaT']
df2['ballVelInWorldCam'] = np.abs(np.sqrt(df2['ballVelInWorldCam_az'].diff()**2  + df2['ballVelInWorldCam_el'].diff()**2) / df2['deltaT'])

In [10]:

startTimeS = 3
endTimeS = startTimeS + 1*6

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


vel = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['velInWorldCam'][startIndex:endIndex],
    name = 'velInWorldCam_az',
    marker_color = colors_idx[0],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


bVel = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ballVelInWorldCam'][startIndex:endIndex],
    name = 'ballVel',
    marker_color = colors_idx[1],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


####


fig.append_trace(eih_el, row=1, col=1)
fig.append_trace(eih_az, row=1, col=1)
fig.append_trace(vel, row=2, col=1)

fig.append_trace(bEih_el, row=1, col=1)
fig.append_trace(bEih_az, row=1, col=1)
fig.append_trace(bVel, row=2, col=1)


Plotting from index: 661 to 1985


In [11]:
yLim=[-50,50]
width=800
height=800
inline=True

layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
)

fig.update_yaxes(range=[-30, 80], row=1, col=1)
fig.update_yaxes(range=[0, 200], row=2, col=1)

# fig.update_xaxes(range=[startTimeS,startTimeS+0.5], row=2, col=1)

fig.update_yaxes(title = 'por (deg)',row=1,col=1)
fig.update_yaxes(title = 'vel (deg/s)',row=2,col=1)
fig.update_xaxes(title = 'time (s)',row=3,col=1)

fig.update_layout(legend_orientation="h", 
             xaxis2_rangeslider_visible=True, xaxis2_rangeslider_thickness=0.1,
                  xaxis2_rangeslider_range=[startTimeS,startTimeS+0.5])

fig.update_layout(height=height, width=width, title_text="Stacked Subplots")

iplot(fig)



In [12]:


startTimeS = 3
endTimeS = startTimeS + 1*6

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


vel = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['velInWorldCam'][startIndex:endIndex],
    name = 'velInWorldCam_az',
    marker_color = colors_idx[0],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


bVel = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ballVelInWorldCam'][startIndex:endIndex],
    name = 'ballVel',
    marker_color = colors_idx[1],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


####


fig.append_trace(eih_el, row=1, col=1)
fig.append_trace(eih_az, row=1, col=1)
fig.append_trace(vel, row=2, col=1)

fig.append_trace(bEih_el, row=1, col=1)
fig.append_trace(bEih_az, row=1, col=1)
fig.append_trace(bVel, row=2, col=1)



Plotting from index: 661 to 1985


new strategy!
Let's edit the trial info file to focus on the final calibration assessment
Debug the track there, where the az / el should be very predictable, and comparable to a ground truth

In [13]:
df2 = interpRaw

ballAzEl = df2.apply(lambda arbitraryRowName: calcAzEl(arbitraryRowName,'ballPos'),axis=1)
df2['ball_az'],df2['ball_el'] = zip(*ballAzEl)

azEl = df2.apply(lambda arbitraryRowName: calcAzEl(arbitraryRowName,'gaze_point_3d'),axis=1)
df2['gazeInWorldCam_az'],df2['gazeInWorldCam_el'] = zip(*azEl)

df2['deltaT'] = df2['worldVidTimeSec'].diff()

df2['velInWorldCam_az'] = df2['gazeInWorldCam_az'].diff() / df2['deltaT']
df2['velInWorldCam_el'] = df2['gazeInWorldCam_el'].diff() / df2['deltaT']
df2['velInWorldCam'] = np.abs(np.sqrt(df2['gazeInWorldCam_az'].diff()**2  + df2['gazeInWorldCam_el'].diff()**2) / df2['deltaT'])

df2['ballVelInWorldCam_az'] = df2['ball_az'].diff() / df2['deltaT']
df2['ballVelInWorldCam_el'] = df2['ball_el'].diff() / df2['deltaT']
df2['ballVelInWorldCam'] = np.abs(np.sqrt(df2['ballVelInWorldCam_az'].diff()**2  + df2['ballVelInWorldCam_el'].diff()**2) / df2['deltaT'])